<a href="https://colab.research.google.com/github/MRK4863/healthcare_chat_asst/blob/main/google_colab/graphRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.9 MB/s eta 0:00:00


In [9]:
import pandas as pd
from neo4j import GraphDatabase
from itertools import combinations

In [10]:
# Read from environment variables or .env file
uri = "neo4j+s://fecbe539.databases.neo4j.io"
username = "neo4j"
password = "yaxMZJyX9a6Ph4SNdRncWJWbSWbmiNGS9Flah_UV100"

In [11]:
# Connect to Neo4j Aura
driver = GraphDatabase.driver(
    uri,
    auth=("neo4j", password)
)

In [12]:
df_drug_db = pd.read_csv("/content/drugbank_clean.csv")

filtered_df = df_drug_db.dropna(subset=['name', 'drugbank-id'])

name_to_drugbank_dict = dict(zip(filtered_df['name'], filtered_df['drugbank-id']))

<ipython-input-12-3289901911>:1: DtypeWarning: Columns (29,30,31,32,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_drug_db = pd.read_csv("/content/drugbank_clean.csv")


In [13]:
def get_drug_info(tx, drug_names):
    query = """
    MATCH (d:Drug)
    WHERE d.name IN $drug_names
    RETURN d.drugbank_id AS id, d.name AS name, d.description AS description,
           d.indication AS indication, d.mechanism_of_action AS mechanism, d.toxicity AS toxicity, d.food_interactions as food_interactions
    """
    return list(tx.run(query, drug_names=drug_names))

def get_interactions(tx, pairs):
    results = []
    for drug1, drug2 in pairs:
        # drug_1_id, drug_2_id = name_to_drugbank_dict[drug1], name_to_drugbank_dict[drug2]
        query = """
        MATCH (a:Drug {name: $drug1})-[r:INTERACTS_WITH]-(b:Drug {name: $drug2})
        RETURN a.name AS drug1, b.name AS drug2, r.description AS description
        """
        rec = tx.run(query, drug1=drug1, drug2=drug2).single()
        if rec:
            results.append(dict(rec))
    return results

drug_list = ["Titanium dioxide", "Verteporfin", "Trioxsalen", "Cetirizine", "Acetaminophen"]
pairs = list(combinations(drug_list, 2))

with driver.session() as session:
    drug_infos = session.read_transaction(get_drug_info, drug_list)
    interaction_infos = session.read_transaction(get_interactions, pairs)

<ipython-input-13-4113845751>:27: DeprecationWarning: read_transaction has been renamed to execute_read
  drug_infos = session.read_transaction(get_drug_info, drug_list)
<ipython-input-13-4113845751>:28: DeprecationWarning: read_transaction has been renamed to execute_read
  interaction_infos = session.read_transaction(get_interactions, pairs)


In [21]:
def get_drugInfo_userPrompt(drug_infos, interaction_infos):
    """
    Build a well-organized, markdown-formatted prompt for drug information analysis.

    Args:
        drug_infos (list): List of dictionaries containing drug information
        interaction_infos (list): List of dictionaries containing drug interaction information

    Returns:
        str: Formatted prompt string
    """

    prompt = """# Clinical Drug Analysis Assistant

## Instructions
You are a clinical assistant tasked with analyzing drug information. Please follow these steps:

1. **Summarize each drug** - Provide a clear, concise summary of each medication
2. **Analyze combination risks** - List and explain the risks of combining any of these drugs
3. **Base responses on provided data only** - Do not generate information that cannot be verified from the data below

---

## Drug Information Database

"""

    # Add individual drug information
    for i, drug in enumerate(drug_infos, 1):
        prompt += f"""### {i}. {drug['name']}

**Description:** {drug['description']}

**Clinical Details:**
- **Indication:** {drug['indication']}
- **Mechanism of Action:** {drug['mechanism']}
- **Toxicity Profile:** {drug['toxicity']}
- **Food Interactions:** {drug['food_interactions']}

---

"""

    # Add drug interaction information
    prompt += """## Drug-Drug Interaction Analysis

"""

    if interaction_infos and len(interaction_infos) > 0:
        prompt += """### Known Interactions

"""
        for i, interaction in enumerate(interaction_infos, 1):
            prompt += f"""**{i}. {interaction['drug1']} + {interaction['drug2']}**
- Risk: {interaction['description']}

"""
    else:
        prompt += """### No Known Interactions
No documented drug-drug interactions were found among the provided medications based on the available data.

"""

    prompt += """---

## Output Requirements

Please structure your response as follows:

### Drug Summaries
Provide a concise summary for each medication listed above.

### Combination Risk Assessment
Analyze and explain any risks associated with combining these medications, based solely on the interaction data provided.

### Clinical Recommendations
Offer appropriate guidance for patients or clinicians based on the available information. Include food-interaction recommendations as well if appropriate

**Note:** All information should be clearly attributed to the provided data sources. Do not include speculative or unverified information."""

    return prompt

prompt = get_drugInfo_userPrompt(drug_infos, interaction_infos)


In [22]:
print(prompt)

# Clinical Drug Analysis Assistant

## Instructions
You are a clinical assistant tasked with analyzing drug information. Please follow these steps:

1. **Summarize each drug** - Provide a clear, concise summary of each medication
2. **Analyze combination risks** - List and explain the risks of combining any of these drugs
3. **Base responses on provided data only** - Do not generate information that cannot be verified from the data below

---

## Drug Information Database

### 1. Acetaminophen

**Description:** Acetaminophen (paracetamol), also commonly known as _Tylenol_, is the most commonly taken analgesic worldwide and is recommended as first-line therapy in pain conditions by the World Health Organization (WHO).[A176318] It is also used for its antipyretic effects, helping to reduce fever.[F4124] This drug was initially approved by the U.S. FDA in 1951 and is available in a variety of forms including syrup form, regular tablets, effervescent tablets, injection, suppository, and ot

In [53]:
sample_messages=[
        {"role": "system", "content": "You are a helpful clinical assistant."},
        {"role": "user", "content": prompt}
    ]

sample_response = get_sambanova_response(sample_messages, model = "Meta-Llama-3.3-70B-Instruct")

print(sample_response)

SyntaxError: invalid syntax (<ipython-input-53-2771570239>, line 1)

In [39]:
from openai import OpenAI
import openai
import base64
import requests
import json

def get_sambanova_response(messages, model = "Meta-Llama-3.1-8B-Instruct"):

    client = OpenAI(
        api_key="f9c890ca-64fa-4e37-ab62-fd9a1e6c4de6",
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.1,
        # top_p=0.1
    )

    return response.choices[0].message.content

def append_message_to_list(messages, role, content):
    messages.append({"role":role, "content": content})

def encode_image_from_path(image_path: str) -> str:
    """Encode image from local file path to base64 string"""

    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def encode_image_from_url(image_url: str) -> str:
    """Download and encode image from URL to base64 string"""
    # Download the image
    response = requests.get(image_url, timeout=30)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Encode to base64
    image_base64 = base64.b64encode(response.content).decode('utf-8')
    return image_base64

def encode_image(image_source: str) -> str:
    """
    Universal function to encode image from either local path or URL

    Args:
        image_source: Either a local file path or HTTP/HTTPS URL

    Returns:
        Base64 encoded string of the image
    """
    if image_source.startswith(('http://', 'https://')):
        return encode_image_from_url(image_source)
    else:
        return encode_image_from_path(image_source)

In [40]:
def get_drugListExtractor_systemPrompt():
    prompt = """# Drug Name Extraction System Prompt

```
You are a medical AI assistant specialized in extracting drug names from text and images.

Your task is to identify and extract all drug names (generic and brand names) mentioned in the provided content.

INSTRUCTIONS:
1. Extract ALL drug names, including:
   - Generic drug names (e.g., acetaminophen, ibuprofen)
   - Brand names (e.g., Tylenol, Advil)
   - Prescription medications
   - Over-the-counter medications
   - Supplements and vitamins with specific drug names

2. DO NOT include:
   - General terms like "medication", "pills", "tablets"
   - Dosage information (e.g., "500mg")
   - Medical conditions or symptoms
   - Non-drug substances unless they are medications

3. Return your response in the following JSON format ONLY:
{
    "drug_names": ["drug1", "drug2", "drug3"],
    "confidence": [0.95, 0.87, 0.92]
}

4. If no drug names are found, return:
{
    "drug_names": [],
    "confidence": []
}

5. Confidence scores should be between 0.0 and 1.0, where:
   - 1.0 = Absolutely certain it's a drug name
   - 0.8-0.9 = Very confident
   - 0.6-0.7 = Moderately confident
   - Below 0.6 = Low confidence (consider excluding)

IMPORTANT: Return ONLY the JSON response, no additional text or explanations.
```"""
    return prompt

In [50]:


# client = OpenAI(
#     api_key="f9c890ca-64fa-4e37-ab62-fd9a1e6c4de6",
#     base_url="https://api.sambanova.ai/v1",
# )

# response = client.chat.completions.create(
#     model="Meta-Llama-3.3-70B-Instruct",
#     messages=[
#         {"role": "system", "content": "You are a helpful clinical assistant."},
#         {"role": "user", "content": prompt}
#     ],
#     temperature=0.1,
#     top_p=0.1
# )

# print(response.choices[0].message.content)



extract_messages = [
        {
            "role": "system",
            "content": get_drugListExtractor_systemPrompt()
        }
    ]

choice = input("Enter your preference:\n1. Enter drug-names as text\n2. Upload drug-names as image\n:")
if choice == "1":
    extract_messages += [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": input()}
            ]
        }
    ]
elif choice == "2":
    image_source = input("Enter the image source: ")
    image_base64 = encode_image(image_source)
    extract_messages += [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Extract the drug-names from the image"},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}}
            ]
        }
    ]
# print(json.dumps(extract_messages, indent = 2))
extract_response = get_sambanova_response(extract_messages, model = "Llama-4-Maverick-17B-128E-Instruct")
print(extract_response)

Enter your preference:
1. Enter drug-names as text
2. Upload drug-names as image
:2
Enter the image source: /content/image.png
{
    "drug_names": ["Lepirudin", "Cetuximab", "Dornase alfa", "Denileukin diftitox", "Etanercept", "Bivalirudin", "Leuprolide", "Peginterferon alfa-2a", "Alteplase", "Sermorelin", "Interferon alfa-n1", "Darbepoetin alfa", "Urokinase", "Goserelin", "Reteplase", "Erythropoietin", "Salmon calcitonin", "Interferon alfa-n3", "Pegfilgrastim", "Sargramostim"],
    "confidence": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}


In [61]:
extracted_drug_names = json.loads(extract_response[extract_response.find("{"): extract_response.rfind("}")+1])["drug_names"]

In [62]:
print(extracted_drug_names)

['Lepirudin', 'Cetuximab', 'Dornase alfa', 'Denileukin diftitox', 'Etanercept', 'Bivalirudin', 'Leuprolide', 'Peginterferon alfa-2a', 'Alteplase', 'Sermorelin', 'Interferon alfa-n1', 'Darbepoetin alfa', 'Urokinase', 'Goserelin', 'Reteplase', 'Erythropoietin', 'Salmon calcitonin', 'Interferon alfa-n3', 'Pegfilgrastim', 'Sargramostim']


In [65]:
def complete_graphrag_search(extracted_drug_names_list: list[str]):
    drug_list = extracted_drug_names_list
    pairs = list(combinations(drug_list, 2))

    with driver.session() as session:
        drug_infos = session.read_transaction(get_drug_info, drug_list)
        interaction_infos = session.read_transaction(get_interactions, pairs)

    return drug_infos, interaction_infos

def get_responder_output(extracted_drug_names_list: list[str]):
    drug_infos, interaction_infos = complete_graphrag_search(extracted_drug_names_list)
    user_prompt = get_drugInfo_userPrompt(drug_infos, interaction_infos)
    messages=[
        {"role": "system", "content": "You are a helpful clinical assistant."},
        {"role": "user", "content": user_prompt}
    ]

    response = get_sambanova_response(messages, model = "Meta-Llama-3.3-70B-Instruct")

    return response

In [66]:
response = get_responder_output(extracted_drug_names)

<ipython-input-65-2980986799>:6: DeprecationWarning: read_transaction has been renamed to execute_read
  drug_infos = session.read_transaction(get_drug_info, drug_list)
<ipython-input-65-2980986799>:7: DeprecationWarning: read_transaction has been renamed to execute_read
  interaction_infos = session.read_transaction(get_interactions, pairs)


In [67]:
print(response)

### Drug Summaries

1. **Lepirudin**: No information available.
2. **Cetuximab**: A recombinant chimeric human/mouse IgG1 monoclonal antibody used for the treatment of head and neck cancer and metastatic colorectal cancer. It binds to epidermal growth factor receptor (EGFR) and inhibits the binding of epidermal growth factor (EGF).
3. **Dornase alfa**: A biosynthetic form of human deoxyribonuclease I (DNase I) enzyme used as adjunct therapy in the treatment of cystic fibrosis. It cleaves extracellular DNA, reducing sputum viscosity.
4. **Denileukin diftitox**: A recombinant DNA-derived cytotoxic protein for the treatment of cutaneous T-cell lymphoma. It binds to the high-affinity IL-2 receptor complex, selectively killing IL-2 bearing cells.
5. **Etanercept**: A dimeric fusion protein consisting of the extracellular ligand-binding portion of the human 75 kilodalton (p75) tumor necrosis factor receptor (TNFR) linked to the Fc portion of human IgG1. Used to treat rheumatoid arthritis, ps

In [68]:
filtered_df["name"].head(20)

NameError: name 'df' is not defined